# Fix Ransom Seed

A random seed (or seed state, or just seed) is a number (or vector) used to **initialize** a pseudorandom number generator\
So if the execution sequence after setting the seed is the same, the random numbers sequence will be the same.\
Each run of the cell will generate the new random number (a new step in the execution sequence), even though its the same cell.\
So if you want the cell to generate the exact same number, you need to put the seed fixing code into that cell. So that each run of that cell will initialize the seed to the same value.

In [ ]:
import torch
torch.manual_seed(0)
import random
random.seed(0)
import numpy as np
np.random.seed(0)

# Tutorial

In [1]:
%matplotlib inline

In [2]:
import torch

In [3]:
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)  # torch.tensor has requires_grad=False
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b  # matmul(): 1-dim x is prepended to its dimension for the purpose of the matrix multiply. After the matrix multiply, the prepended dimension is removed. {math: (x**T * w)**T}
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print(loss)

tensor(1.3513, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


In [4]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7f14ea0b1790>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x7f14ea0b17d0>


In [5]:
# --- If run backward() second time. Set all the gradient of leaf tensor to 0 firt, or else grad will be accumulated --- #
if w.grad != None and b.grad != None:                                                                                    #
    w.grad.zero_()                                                                                                       #
    b.grad.zero_()                                                                                                       # 
# ---------- No need for above tow line, if only backwar() once. Because no grad before calling backward() yet --------- #
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2444, 0.2308, 0.2628],
        [0.2444, 0.2308, 0.2628],
        [0.2444, 0.2308, 0.2628],
        [0.2444, 0.2308, 0.2628],
        [0.2444, 0.2308, 0.2628]])
tensor([0.2444, 0.2308, 0.2628])


In [6]:
print(z.requires_grad)
print(z.grad)  # None: Although z.requires_grad=True, it is not a leaf Tensor. So its .grad attribute won't be populated during autograd.backward()

True
None


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  


In [7]:
# New calculation (from leaf tensor to backwarded tensor), build new graph, can call backward again no matter how
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print(loss)

tensor(1.3513, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


In [8]:
w.grad.zero_()
b.grad.zero_()
loss.backward(retain_graph=True)  # Specify retain_graph=True, then backward() on same graph (no new claculation) can be called one more time
print(w.grad)
print(b.grad)

tensor([[0.2444, 0.2308, 0.2628],
        [0.2444, 0.2308, 0.2628],
        [0.2444, 0.2308, 0.2628],
        [0.2444, 0.2308, 0.2628],
        [0.2444, 0.2308, 0.2628]])
tensor([0.2444, 0.2308, 0.2628])


In [9]:
w.grad.zero_()
b.grad.zero_()
loss.backward()  # Default retain_graph=None, then backward() on same graph (no new claculation) cannot be called
print(w.grad)
print(b.grad)

tensor([[0.2444, 0.2308, 0.2628],
        [0.2444, 0.2308, 0.2628],
        [0.2444, 0.2308, 0.2628],
        [0.2444, 0.2308, 0.2628],
        [0.2444, 0.2308, 0.2628]])
tensor([0.2444, 0.2308, 0.2628])


In [10]:
loss.backward()  # RuntimeError: Trying to backward through the graph a second time

RuntimeError: ignored

In [11]:
z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)
print(w.requires_grad, b.requires_grad)  # tensor not calculated by in no_grad() block, won't be affected

True
False
True True


In [12]:
z = torch.matmul(x, w) + b
print(z.requires_grad)
z_det = z.detach()  # detach(): Returns a new Tensor, detached from the current graph. The result will never require gradient.
print(z_det.requires_grad)
print(z.requires_grad)  # Not affected

True
False
True


In [13]:
inp = torch.eye(5, requires_grad=True)  # same as inp=torch.flatten(torch.eye(5)).requires_grad_(), but result being reshaped to the size
out = (inp + 1).pow(2)
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n {inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n {inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n {inp.grad}")

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


In [14]:
# For details,see https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#optional-reading-vector-calculus-using-autograd
inp = torch.tensor([1.,0.,0.], requires_grad=True)
print(inp)
print(f"\nJ = \n{torch.tensor([[4, 0, 0], [0, 2, 0], [0, 0, 2]])}\n")
out = (inp + 1).pow(2)
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n {inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n {inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n {inp.grad}")

tensor([1., 0., 0.], requires_grad=True)

J = 
tensor([[4, 0, 0],
        [0, 2, 0],
        [0, 0, 2]])

First call
 tensor([4., 2., 2.])

Second call
 tensor([8., 4., 4.])

Call after zeroing gradients
 tensor([4., 2., 2.])
